In [2]:
import fitz

def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document[page_num]
        text += page.get_text()
    document.close()
    return text.strip()

pdf_text = extract_text_from_pdf("report.pdf")
print(f"Extracted text (first 500 chars): {pdf_text[:500]}")


BP:190,
Blood Sugar:34,
Heart Rate:90


In [7]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

# Initialize the encoder and Qdrant client
encoder = SentenceTransformer("all-MiniLM-L6-v2")
client = QdrantClient(":memory:")  # For in-memory use; replace with cluster host info for persistent storage


collection_name = "medical_reports"
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by the encoder
        distance=models.Distance.COSINE,  # Use cosine similarity
    ),
)

def chunk_text(text, chunk_size=500):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

chunks = chunk_text(pdf_text)

client.upload_points(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(chunk).tolist(),
            payload={"chunk": chunk}
        )
        for idx, chunk in enumerate(chunks)
    ],
)

print(f"Stored {len(chunks)} chunks in Qdrant.")


Stored 1 chunks in Qdrant.


In [8]:
def query_relevant_chunks(query_text, top_k=5):
    query_vector = encoder.encode(query_text).tolist()
    hits = client.query_points(
        collection_name=collection_name,
        query=query_vector,
        limit=top_k,
    ).points
    return [hit.payload["chunk"] for hit in hits]

query = "summarize the medical findings"
relevant_chunks = query_relevant_chunks(query)

print("Relevant chunks for summarization:")
for chunk in relevant_chunks:
    print(chunk[:200], "...")


Relevant chunks for summarization:
BP:190,
Blood Sugar:34,
Heart Rate:90 ...


In [15]:
from openai import OpenAI
from settings import *
import os


generation_model_name : str


temperature:float= 0.9
top_p=0.9
max_tokens:int= 2048
stream: bool= True
llm_name:str ="Meta-Llama"



monster_client = OpenAI(
            base_url="https://llm.monsterapi.ai/v1/",
            api_key=str(MONSTER_API_KEY))


monster_ai_model_name = {
                "Google-Gemma": "google/gemma-2-9b-it",
                "Mistral": "mistralai/Mistral-7B-Instruct-v0.2",
                "Microsoft-Phi": "microsoft/Phi-3-mini-4k-instruct",
                "Meta-Llama": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            }

message=[
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'\''m quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'\''m cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}]



response  = monster_client.chat.completions.create(
    model=monster_ai_model_name[llm_name],
    messages=message,
    temperature = temperature,
    top_p = top_p,
    max_tokens = max_tokens,
    stream=stream
)


generated_text = ""
for chunk in response:
    if chunk.choices[0].delta.content is not None:
        generated_text += chunk.choices[0].delta.content
        

print(generated_text)



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


I can provide you with some classic and creative mayonnaise recipes. Here are a few:

**Classic Mayonnaise Recipe:**

Ingredients:

- 2 large egg yolks
- 1 tablespoon Dijon mustard
- 1 tablespoon white wine vinegar or lemon juice
- 1/2 cup (120 ml) neutral-tasting oil, such as canola or grapeseed
- Salt, to taste

Instructions:

1. In a medium bowl, whisk together egg yolks, mustard, and vinegar until smooth.
2. Slowly pour the oil into the bowl while continuously whisking the mixture.
3. Continue whisking until the mixture thickens and emulsifies.
4. Season with salt to taste.

**Garlic Mayonnaise (Aioli) Recipe:**

Ingredients:

- 2 large egg yolks
- 1 clove garlic, minced
- 1 tablespoon lemon juice
- 1/2 cup (120 ml) neutral-tasting oil, such as canola or grapeseed
- Salt, to taste

Instructions:

1. In a medium bowl, whisk together egg yolks, garlic, and lemon juice until smooth.
2. Slowly pour the oil into the bowl while continuously whisking the mixture.
3. Continue whisking unti